In [15]:
import pandas as pd
import numpy as np

import plotly.express as px

from collections import Counter

In [16]:
raw_data = pd.read_csv("datasets/raw_dataset.csv")
raw_data.head()

,author,author_avatar_url,membership_date,author_bio,num_followers,v3_newsletter_subs,date,reading_time,title,post_url,claps,num_unique_clappers,image_url,num_responses,tags,author_url
0,Tim Denning,https://miro.medium.com/1*bfllCILGW4yHKXgFo8Jk...,2021-10-07 13:55:47,Aussie Blogger with 500M+ views — Writer for C...,314216,3373,2023-03-24 06:32:53.325000,4.0,The Idea of Acting “Professional” Is an Absolu...,https://medium.com/swlh/the-idea-of-acting-pro...,1276,116,https://miro.medium.com/1*usQlrBFDwBslO2aUmbTn...,23,"['business', 'startup', 'life', 'advice', 'soc...",https://timdenning.medium.com/
1,Tim Denning,https://miro.medium.com/1*bfllCILGW4yHKXgFo8Jk...,2021-10-07 13:55:47,Aussie Blogger with 500M+ views — Writer for C...,314216,3373,2023-03-23 00:01:35.049000,5.0,Build a Thriving Community to Drastically Enha...,https://writingcooperative.com/build-a-thrivin...,1725,81,https://miro.medium.com/1*Om2sLjT-DYklcoSSSpIc...,21,"['social-media', 'marketing', 'entrepreneurshi...",https://timdenning.medium.com/
2,Tim Denning,https://miro.medium.com/1*bfllCILGW4yHKXgFo8Jk...,2021-10-07 13:55:47,Aussie Blogger with 500M+ views — Writer for C...,314216,3373,2023-03-22 12:31:32.538000,3.0,The 4-Minute Summary of the US Economy over th...,https://medium.com/swlh/the-4-minute-summary-o...,652,77,https://miro.medium.com/1*0vzjFhoYnHt2ExGgfKX3...,10,"['economy', 'money', 'investing', 'business', ...",https://timdenning.medium.com/
3,Tim Denning,https://miro.medium.com/1*bfllCILGW4yHKXgFo8Jk...,2021-10-07 13:55:47,Aussie Blogger with 500M+ views — Writer for C...,314216,3373,2023-03-21 22:04:26.792000,3.0,Self-Education Gets You Further in Life than a...,https://medium.com/mind-cafe/self-education-ge...,681,44,https://miro.medium.com/1*1TuSZwPdoTrgmJk3dlNY...,15,"['education', 'self-improvement', 'learning', ...",https://timdenning.medium.com/
4,Tim Denning,https://miro.medium.com/1*bfllCILGW4yHKXgFo8Jk...,2021-10-07 13:55:47,Aussie Blogger with 500M+ views — Writer for C...,314216,3373,2023-03-21 16:32:08.431000,3.0,The One-Person Business Model Is a Lie,https://entrepreneurshandbook.co/the-one-perso...,1282,90,https://miro.medium.com/1*QUBY4Syx8eGExkJ5wxBn...,23,"['startup', 'solopreneur', 'entrepreneurship',...",https://timdenning.medium.com/


In [22]:
articles = raw_data["author"].value_counts().copy()
articles

Tim Denning          2561
Desiree Peralta       395
Colin Horgan          165
Thanos                114
Hasan Aboul Hasan      24
Name: author, dtype: int64

In [41]:
authors = (
    raw_data.loc[:, ["author", "membership_date", "author_bio", "num_followers", "v3_newsletter_subs", "author_url"]]
    .drop_duplicates()
    .reset_index(drop=True)
)
authors["num_articles"] = raw_data["author"].value_counts().values
authors.set_index("author", inplace=True)
authors

,membership_date,author_bio,num_followers,v3_newsletter_subs,author_url,num_articles
author,,,,,,
Tim Denning,2021-10-07 13:55:47,Aussie Blogger with 500M+ views — Writer for C...,314216,3373,https://timdenning.medium.com/,2561
Hasan Aboul Hasan,2022-04-19 16:20:38,"A combination of Human, Father, Developer, You...",7340,539,https://hasanaboulhasan.medium.com/,395
Colin Horgan,2019-01-04 19:02:39,writer.,21567,32,https://cfhorgan.medium.com/,165
Desiree Peralta,2022-06-01 13:46:54,Turning ideas into reality. Programmer by prof...,21955,264,https://dessyperalt.medium.com/,114
Thanos,2022-11-25 20:18:07,Soon to be MD. Here to make your life better o...,1959,105,https://anonwit.medium.com/,24


In [71]:
# average claps per article
authors["average_claps"] = raw_data.groupby("author")["claps"].mean()
# average unique claps per article
authors["average_unique_clappers"] = raw_data.groupby("author")["num_unique_clappers"].mean()
# average reading time
authors["average_reading_time"] = raw_data.groupby("author")["reading_time"].mean()
# average number of responses
authors["average_responses"] = raw_data.groupby("author")["num_responses"].mean()

In [72]:
tag_counts = raw_data.groupby("author").tags.apply(sum).apply(lambda x: eval(x.replace("][", ""))).apply(Counter).apply(dict).reset_index()
num_tags = tag_counts["tags"].apply(lambda x: Counter(x.values()))

for idx, author in enumerate(tag_counts.author):
    print(author)
    print(num_tags[idx])

Colin Horgan
Counter({1: 220, 2: 21, 3: 17, 4: 6, 29: 2, 5: 2, 7: 2, 39: 1, 17: 1, 28: 1, 14: 1, 6: 1, 10: 1, 18: 1, 12: 1})
Desiree Peralta
Counter({1: 239, 2: 44, 3: 13, 4: 12, 5: 9, 7: 3, 14: 3, 69: 2, 13: 2, 30: 2, 6: 2, 9: 2, 26: 1, 185: 1, 78: 1, 85: 1, 37: 1, 33: 1, 34: 1, 93: 1, 11: 1, 19: 1, 10: 1, 44: 1, 80: 1, 28: 1, 22: 1, 20: 1})
Hasan Aboul Hasan
Counter({1: 60, 2: 11, 3: 3, 6: 1})
Thanos
Counter({1: 189, 2: 17, 3: 17, 4: 5, 6: 4, 10: 3, 9: 2, 12: 2, 5: 2, 20: 1, 16: 1, 7: 1, 13: 1})
Tim Denning
Counter({1: 701, 2: 167, 3: 69, 4: 43, 5: 29, 6: 27, 7: 18, 8: 11, 14: 11, 9: 10, 12: 8, 11: 7, 13: 6, 24: 5, 17: 5, 16: 4, 36: 4, 25: 3, 22: 3, 10: 3, 37: 2, 15: 2, 26: 2, 28: 2, 18: 2, 19: 2, 185: 1, 474: 1, 890: 1, 40: 1, 129: 1, 553: 1, 211: 1, 271: 1, 50: 1, 959: 1, 714: 1, 69: 1, 273: 1, 131: 1, 167: 1, 43: 1, 276: 1, 141: 1, 104: 1, 240: 1, 47: 1, 67: 1, 78: 1, 98: 1, 48: 1, 147: 1, 229: 1, 21: 1, 33: 1, 75: 1, 73: 1, 23: 1, 35: 1})


In [73]:
tag_counts

,author,tags
0,Colin Horgan,"{'technology': 39, 'social-media': 17, 'tik-to..."
1,Desiree Peralta,"{'money': 26, 'finance': 69, 'life-lessons': 1..."
2,Hasan Aboul Hasan,"{'chatgpt': 1, 'gpt-4': 1, 'openai': 1, 'ai': ..."
3,Thanos,"{'personal-development': 2, 'self-improvement'..."
4,Tim Denning,"{'business': 185, 'startup': 474, 'life': 890,..."


In [74]:
def filter_tags(row):
    author, tags_dict = row.values.tolist()
    num_articles = authors.loc[author]["num_articles"]
    ratio = 0.1
    lower_bound = np.ceil(num_articles * ratio)
    return [author, {k: v for k, v in tags_dict.items() if v > lower_bound}]

In [75]:
authors_list = []
tags = []
for idx, row in tag_counts.apply(filter_tags, axis=1):
    authors_list.append(idx)
    tags.append(row)

df = pd.DataFrame({"author": authors_list, "top10pr_tags": tags})
df.set_index("author", inplace=True)
authors["top10pr_tags"] = df["top10pr_tags"].copy()
authors

,membership_date,author_bio,num_followers,v3_newsletter_subs,author_url,num_articles,average_claps,average_unique_clappers,average_reading_time,average_responses,top10pr_tags
author,,,,,,,,,,,
Tim Denning,2021-10-07 13:55:47,Aussie Blogger with 500M+ views — Writer for C...,314216,3373,https://timdenning.medium.com/,2561,2496.384615,350.954315,4.846154,21.811792,"{'startup': 474, 'life': 890, 'entrepreneurshi..."
Hasan Aboul Hasan,2022-04-19 16:20:38,"A combination of Human, Father, Developer, You...",7340,539,https://hasanaboulhasan.medium.com/,395,1164.166667,162.625000,3.708333,32.166667,{}
Colin Horgan,2019-01-04 19:02:39,writer.,21567,32,https://cfhorgan.medium.com/,165,1125.090909,151.000000,4.521212,9.636364,"{'technology': 39, 'politics': 28, 'society': ..."
Desiree Peralta,2022-06-01 13:46:54,Turning ideas into reality. Programmer by prof...,21955,264,https://dessyperalt.medium.com/,114,467.594937,45.496203,4.587342,5.486076,"{'money': 26, 'finance': 69, 'life-lessons': 1..."
Thanos,2022-11-25 20:18:07,Soon to be MD. Here to make your life better o...,1959,105,https://anonwit.medium.com/,24,115.815789,10.359649,2.991228,1.807018,"{'self-improvement': 20, 'life-lessons': 10, '..."


In [76]:
authors_final = authors.copy().reset_index()
authors_final

,author,membership_date,author_bio,num_followers,v3_newsletter_subs,author_url,num_articles,average_claps,average_unique_clappers,average_reading_time,average_responses,top10pr_tags
0,Tim Denning,2021-10-07 13:55:47,Aussie Blogger with 500M+ views — Writer for C...,314216,3373,https://timdenning.medium.com/,2561,2496.384615,350.954315,4.846154,21.811792,"{'startup': 474, 'life': 890, 'entrepreneurshi..."
1,Hasan Aboul Hasan,2022-04-19 16:20:38,"A combination of Human, Father, Developer, You...",7340,539,https://hasanaboulhasan.medium.com/,395,1164.166667,162.625000,3.708333,32.166667,{}
2,Colin Horgan,2019-01-04 19:02:39,writer.,21567,32,https://cfhorgan.medium.com/,165,1125.090909,151.000000,4.521212,9.636364,"{'technology': 39, 'politics': 28, 'society': ..."
3,Desiree Peralta,2022-06-01 13:46:54,Turning ideas into reality. Programmer by prof...,21955,264,https://dessyperalt.medium.com/,114,467.594937,45.496203,4.587342,5.486076,"{'money': 26, 'finance': 69, 'life-lessons': 1..."
4,Thanos,2022-11-25 20:18:07,Soon to be MD. Here to make your life better o...,1959,105,https://anonwit.medium.com/,24,115.815789,10.359649,2.991228,1.807018,"{'self-improvement': 20, 'life-lessons': 10, '..."


In [78]:
authors_final.to_csv("datasets/authors_processed.csv", index=False)